In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from tensorflow.keras.layers import Conv2D, Dropout, Dense, Flatten, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight

In [3]:
!unzip /content/drive/MyDrive/Mini_Project/archive.zip -d /content/aqua_trash/

Archive:  /content/drive/MyDrive/Mini_Project/archive.zip
  inflating: /content/aqua_trash/Images/000000_JPG.rf.d3371cb3d63a59c5ba6730368b7905af.jpg  
  inflating: /content/aqua_trash/Images/000000_jpg.rf.beffaf3b548106ccf1da5dc629bc9504.jpg  
  inflating: /content/aqua_trash/Images/000000_jpg.rf.e662cb85f63817325956fea222d0990f.jpg  
  inflating: /content/aqua_trash/Images/000000_jpg.rf.ee75fdf06813399a8376c6ff7056423a.jpg  
  inflating: /content/aqua_trash/Images/000001_JPG.rf.ccfdd243a78a6cfa119be0349d18c0ed.jpg  
  inflating: /content/aqua_trash/Images/000001_jpg.rf.3b10b06a0479e6c9e52202033c95d203.jpg  
  inflating: /content/aqua_trash/Images/000001_jpg.rf.42c4ba88119aae9d425f5c004cb2ce0d.jpg  
  inflating: /content/aqua_trash/Images/000001_jpg.rf.a23b83dbda863601c92725ae9ba3bab7.jpg  
  inflating: /content/aqua_trash/Images/000002_jpg.rf.ba12a4ce95a3dd81818a5e373efe28be.jpg  
  inflating: /content/aqua_trash/Images/000002_jpg.rf.cc63f3f198e4bafa011b425d63252066.jpg  
  inflating:

In [4]:
df = pd.read_csv('/content/aqua_trash/annotations.csv')

In [5]:
df

,image_name,x_min,y_min,x_max,y_max,class_name
0,000000_jpg.rf.beffaf3b548106ccf1da5dc629bc9504...,75,278,130,294,metal
1,000000_jpg.rf.beffaf3b548106ccf1da5dc629bc9504...,155,31,181,56,metal
2,000000_JPG.rf.d3371cb3d63a59c5ba6730368b7905af...,840,1122,1568,2074,plastic
3,000000_jpg.rf.e662cb85f63817325956fea222d0990f...,78,97,352,246,plastic
4,000000_jpg.rf.ee75fdf06813399a8376c6ff7056423a...,687,1213,904,1432,plastic
...,...,...,...,...,...,...
464,000128_JPG.rf.eb71904df1426bfc208c7f961ae52d22...,156,1360,1214,2627,paper
465,000129_JPG.rf.eb4bb302dd9580081c49afc5465f58ce...,571,1160,880,1546,metal
466,000129_JPG.rf.eb4bb302dd9580081c49afc5465f58ce...,995,903,1418,1584,paper
467,000129_JPG.rf.eb4bb302dd9580081c49afc5465f58ce...,623,1589,1852,2622,paper


In [6]:
CLASSES = np.unique(df['class_name'])
BASE_PATH = '/content/aqua_trash/Images'
df['class_name'].value_counts()

,count
class_name,
plastic,191
metal,118
paper,116
glass,44


In [10]:
!unzip /content/drive/MyDrive/Mini_Project/Project1.v1i.coco.zip -d /content/project1/

Archive:  /content/drive/MyDrive/Mini_Project/Project1.v1i.coco.zip
 extracting: /content/project1/README.dataset.txt  
 extracting: /content/project1/README.roboflow.txt  
   creating: /content/project1/test/
 extracting: /content/project1/test/_annotations.coco.json  
 extracting: /content/project1/test/image104_jpg.rf.7c0ee236f0f76407d702c9166b265a0c.jpg  
 extracting: /content/project1/test/image107_jpg.rf.42610457741aa420985fb2e758199d4b.jpg  
 extracting: /content/project1/test/image10_jpg.rf.a1ddc4098e37228a4c83495c1fa6f9e4.jpg  
 extracting: /content/project1/test/image118_jpg.rf.a390d86841ada92fac70bda6899ed155.jpg  
 extracting: /content/project1/test/image120_jpg.rf.c2878bf5709bde60a46baa6ee96f5212.jpg  
 extracting: /content/project1/test/image138_jpg.rf.573e5b8c01fc453703cbaa383218325e.jpg  
 extracting: /content/project1/test/image143_jpg.rf.5e3699d15f0b31a5d0be1769048fe66d.jpg  
 extracting: /content/project1/test/image144_jpg.rf.5a36da067d70598436ab0f835d9a5fe9.jpg  
 e

In [13]:
!unzip /content/drive/MyDrive/Mini_Project/PembaTrashData.v4i.coco.zip -d /content/premba/

Archive:  /content/drive/MyDrive/Mini_Project/PembaTrashData.v4i.coco.zip
  inflating: /content/premba/README.dataset.txt  
  inflating: /content/premba/README.roboflow.txt  
   creating: /content/premba/train/
 extracting: /content/premba/train/A00001_JPG.rf.6e423bd04aabdfc1b9fed35b986e6441.jpg  
 extracting: /content/premba/train/A00001_JPG.rf.afabc29d04d891472ef78241918fdcc1.jpg  
 extracting: /content/premba/train/A00001_JPG.rf.fe0699aaf8fadd0cbe8dbe9ecc6ee05f.jpg  
 extracting: /content/premba/train/A00002_JPG.rf.1bcca091aaed8bdb97dffa086b5fbce2.jpg  
 extracting: /content/premba/train/A00002_JPG.rf.619dc56384d215896b4c82807b29dc2e.jpg  
 extracting: /content/premba/train/A00002_JPG.rf.c2de77fecfee10a40e1aeb26138a66a8.jpg  
 extracting: /content/premba/train/A00003_JPG.rf.afa2a9d90adfc19864b6f8adf03e4b85.jpg  
 extracting: /content/premba/train/A00003_JPG.rf.be15a4fc97cc9a4cbd757f77dd7d86c3.jpg  
 extracting: /content/premba/train/A00003_JPG.rf.c86ef9edd6619b73dcc87eafae506545.jpg

In [14]:
!unzip /content/drive/MyDrive/Mini_Project/SANDe.v8i.coco.zip -d /content/sande

Archive:  /content/drive/MyDrive/Mini_Project/SANDe.v8i.coco.zip
  inflating: /content/sande/README.dataset.txt  
  inflating: /content/sande/README.roboflow.txt  
   creating: /content/sande/test/
 extracting: /content/sande/test/DSC05680_jpeg.rf.e261f5ef12995a6af3304ee748267d58.jpg  
 extracting: /content/sande/test/IMG_1131-3_jpg.rf.7926fee4b59ed89f339bfb76191e67c0.jpg  
 extracting: /content/sande/test/IMG_1139-3_jpg.rf.bd142c5caa71e724a7165a4c11c7b1a6.jpg  
 extracting: /content/sande/test/IMG_1862_jpeg.rf.88cbb17e412e7c76cf672ea9073447a1.jpg  
 extracting: /content/sande/test/IMG_1889_jpeg.rf.73ef8827b1e40a6db349c450ebd859da.jpg  
 extracting: /content/sande/test/IMG_1926_jpeg.rf.ff40a0cbe11384539ae0c003fc7e381b.jpg  
 extracting: /content/sande/test/IMG_1928_jpeg.rf.9ea706ee30c67cfffbd2f50290119653.jpg  
 extracting: /content/sande/test/IMG_2753_jpg.rf.e1fd10825ffa13d5604545262c797da9.jpg  
 extracting: /content/sande/test/IMG_2756_jpg.rf.70dd900dab03d7c5065e35800a1abd4f.jpg  
 

In [16]:
!unzip /content/drive/MyDrive/Mini_Project/beach-cleaning-object-detection.v14i.coco.zip -d /content/beach/

Archive:  /content/drive/MyDrive/Mini_Project/beach-cleaning-object-detection.v14i.coco.zip
replace /content/beach/README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/beach/README.dataset.txt  
  inflating: /content/beach/README.roboflow.txt  
 extracting: /content/beach/test/104_jpg.rf.8f37540437c29bb66b406e079f15ced5.jpg  
 extracting: /content/beach/test/116_jpg.rf.52e1b60e541bf7aa2ad416ed1e0639cc.jpg  
 extracting: /content/beach/test/11_jpg.rf.207c7da147f61d40541fc5e9e83e009f.jpg  
 extracting: /content/beach/test/121_jpg.rf.4547d5da5c53f78bbe0511702ec43c13.jpg  
 extracting: /content/beach/test/130_jpg.rf.1089ed05abbe2ada57aafac6cf21584a.jpg  
 extracting: /content/beach/test/131_jpg.rf.9d0fd96a10b1b1efd9fa8bd2981f9b8f.jpg  
 extracting: /content/beach/test/14_jpg.rf.073a32cbc64928e16aef39c5bf2844f1.jpg  
 extracting: /content/beach/test/168_jpg.rf.9629805c4fc4664116d2e9a5341997d8.jpg  
 extracting: /content/beach/test/169_jpg.rf.7cd7746fd6c3aeb5271c

In [17]:
!unzip /content/drive/MyDrive/Mini_Project/dataset.v4i.coco.zip -d /content/data/

Archive:  /content/drive/MyDrive/Mini_Project/dataset.v4i.coco.zip
  inflating: /content/data/README.dataset.txt  
  inflating: /content/data/README.roboflow.txt  
   creating: /content/data/test/
  inflating: /content/data/test/_annotations.coco.json  
 extracting: /content/data/test/image122_jpg.rf.499b3f9123c533213cf75286f1de0a75.jpg  
 extracting: /content/data/test/image142_jpg.rf.d9305fe31be1d687ce8da0e4cba43f25.jpg  
 extracting: /content/data/test/image146_jpg.rf.f0c8a6590420795ca5c687c633627b41.jpg  
 extracting: /content/data/test/image147_jpg.rf.112e6689d4bcfa9e160824cc25761dc1.jpg  
 extracting: /content/data/test/image151_jpg.rf.626f651542bee608289b5f11c4224650.jpg  
 extracting: /content/data/test/image159_jpg.rf.840192051439cac5b38a676ed69da7b9.jpg  
 extracting: /content/data/test/image168_jpg.rf.3af0a2a94a3a05c45e6bee255ae5acb0.jpg  
 extracting: /content/data/test/image171_jpg.rf.6f6398b52621d008f3b36e02566b82f3.jpg  
 extracting: /content/data/test/image19_jpg.rf.307

In [18]:
!unzip /content/drive/MyDrive/Mini_Project/photosfinalisee.v3-photosfinalisee.coco.zip -d /content/photo/

Archive:  /content/drive/MyDrive/Mini_Project/photosfinalisee.v3-photosfinalisee.coco.zip
  inflating: /content/photo/README.dataset.txt  
  inflating: /content/photo/README.roboflow.txt  
   creating: /content/photo/test/
 extracting: /content/photo/test/102_jpg.rf.df9e1e5c9d80fcc9ab4987e70d5ff4fc.jpg  
 extracting: /content/photo/test/106_jpg.rf.fa6e8f71b7c5f4900c4bb85347d3cca9.jpg  
 extracting: /content/photo/test/109_jpg.rf.e950bd8bf02a5ab6915af589ff4a0f58.jpg  
 extracting: /content/photo/test/120_jpg.rf.eb51073d71827ee205ca679aba9c9003.jpg  
 extracting: /content/photo/test/125_jpg.rf.3f0c764b65f548ccc44de248ff25619c.jpg  
 extracting: /content/photo/test/132_jpg.rf.a48ac941fd041ea8d671985ce9428f18.jpg  
 extracting: /content/photo/test/134_jpg.rf.bc80fc71c2607846f0c17f219b6eee60.jpg  
 extracting: /content/photo/test/149_jpg.rf.84d84d4752da1f0822526ff58847eb92.jpg  
 extracting: /content/photo/test/154_jpg.rf.a56861c6e57ecfcf984f5f0540d6580d.jpg  
 extracting: /content/photo/te

#Dataset Preparation

In [37]:
import os

def count_files_in_directory(dir_path, skip_dirs=None):
    if skip_dirs is None:
        skip_dirs = []

    file_count = 0
    dir_structure = {}

    for root, dirs, files in os.walk(dir_path):
        # Skip specified directories
        if any(skip in root for skip in skip_dirs):
            continue

        # Count files in the current directory
        current_count = len(files)
        file_count += current_count

        # Store the directory and its file count
        dir_structure[root] = current_count

    return dir_structure, file_count

# Specify the directory to analyze and directories to skip
base_directory = '/content'
skip_directories = ['/content/sample_data', '/content/drive']

# Get the directory structure and total file count
directory_structure, total_files = count_files_in_directory(base_directory, skip_dirs=skip_directories)

# Print the directory structure and counts
for directory, count in directory_structure.items():
    print(f"{directory}: {count} files")

print(f"\nTotal files in '{base_directory}': {total_files}")

/content: 0 files
/content/.config: 8 files
/content/.config/logs: 0 files
/content/.config/logs/2024.09.20: 6 files
/content/.config/configurations: 1 files
/content/.ipynb_checkpoints: 0 files
/content/data: 2 files
/content/data/train: 441 files
/content/data/valid: 126 files
/content/data/test: 63 files
/content/aqua_trash: 3 files
/content/aqua_trash/Images: 369 files
/content/premba: 2 files
/content/premba/train: 3788 files
/content/wrong data 3: 1 files
/content/wrong data 3/val: 0 files
/content/wrong data 3/val/labels: 133 files
/content/wrong data 3/val/images: 133 files
/content/wrong data 3/train: 0 files
/content/wrong data 3/train/labels: 353 files
/content/wrong data 3/train/images: 353 files
/content/wrong data 2: 1 files
/content/wrong data 2/val: 0 files
/content/wrong data 2/val/labels: 79 files
/content/wrong data 2/val/images: 79 files
/content/wrong data 2/train: 0 files
/content/wrong data 2/train/labels: 290 files
/content/wrong data 2/train/images: 290 files
/

###Sande

In [47]:
import os
import json
!pip install pyyaml
import yaml

# Define the path to the train directory
train_dir = '/content/sande/train'

# Initialize a list to store the unique class names
unique_classes = set()

# Search for the annotation.coco.json file in the directory
for root, dirs, files in os.walk(train_dir):
    for file in files:
        if file == '_annotations.coco.json':
            annotation_file_path = os.path.join(root, file)
            # Load the JSON data
            with open(annotation_file_path, 'r') as f:
                data = json.load(f)
                # Extract class names from categories
                for category in data.get('categories', []):
                    unique_classes.add(category['name'])

# Convert the set to a sorted list
unique_classes_list = sorted(unique_classes)
unique_classes_list

import os
import json

# Define the paths
base_path = '/content'
train_path = os.path.join(base_path, 'train')
valid_path = os.path.join(base_path, 'valid')
test_path = os.path.join(base_path, 'test')

# Function to convert COCO to YOLO format
def coco_to_yolo(coco_json_path, image_dir, output_dir, class_names):
    # Read the COCO JSON file
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Create output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)

    # Mapping of category id to class id
    category_map = {category['id']: idx for idx, category in enumerate(class_names)}

    # Process each image
    for image in coco_data['images']:
        image_id = image['id']
        image_file_name = image['file_name']
        width = image['width']
        height = image['height']

        # Prepare YOLO annotation list
        yolo_annotations = []

        # Find corresponding annotations for this image
        for annotation in coco_data['annotations']:
            if annotation['image_id'] == image_id:
                category_id = annotation['category_id']
                if category_id in category_map:
                    # Get YOLO format (center_x, center_y, width, height)
                    bbox = annotation['bbox']
                    x_center = (bbox[0] + bbox[2] / 2) / width
                    y_center = (bbox[1] + bbox[3] / 2) / height
                    w = bbox[2] / width
                    h = bbox[3] / height
                    class_id = category_map[category_id]
                    yolo_annotations.append(f"{class_id} {x_center} {y_center} {w} {h}")

        # Write YOLO annotations to a text file
        if yolo_annotations:
            with open(os.path.join(output_dir, image_file_name.replace('.jpg', '.txt')), 'w') as out_file:
                out_file.write("\n".join(yolo_annotations))

# Define class names from the provided category info
class_names = [
    {"id": 0, "name": "beach"},
    {"id": 1, "name": "beach"},
    {"id": 2, "name": "bike"},
    {"id": 3, "name": "car"},
    {"id": 4, "name": "crow"},
    {"id": 5, "name": "crowd"},
    {"id": 6, "name": "dog"},
    {"id": 7, "name": "garbage"},
    {"id": 8, "name": "garbage cluster"},
    {"id": 9, "name": "horse"},
    {"id": 10, "name": "obstacle"},
    {"id": 11, "name": "pedestrian"},
    {"id": 12, "name": "plant"},
    {"id": 13, "name": "shop"},
    {"id": 14, "name": "shops"},
    {"id": 15, "name": "slipper"},
]

# Convert each dataset
for dataset in ['train', 'valid', 'test']:
    coco_file_path = os.path.join(base_path, dataset, '_annotations.coco.json')
    image_dir_path = os.path.join(base_path, dataset)
    output_dir_path = os.path.join(base_path, dataset, 'yolo_annotations')

    coco_to_yolo(coco_file_path, image_dir_path, output_dir_path, class_names)

print("Conversion to YOLO format completed.")

import os
import json
import shutil

# Define the base path
base_path = '/content/sande'
sande_yolo_path = os.path.join(base_path, 'sande_yolo')

# Create sande_yolo directory
os.makedirs(sande_yolo_path, exist_ok=True)

# Initialize list to hold all class names
unique_classes = []

# Function to add annotations and images to the sande_yolo directory
def add_to_yolo_dir(dataset):
    dataset_path = os.path.join(base_path, dataset)
    output_yolo_dir = os.path.join(dataset_path, 'yolo_annotations')

    # Load class names from the dataset
    class_names = []
    with open(os.path.join(dataset_path, '_annotations.coco.json'), 'r') as f:
        coco_data = json.load(f)
        for category in coco_data.get('categories', []):
            if category['name'] not in class_names:
                class_names.append(category['name'])

    # Append to unique_classes
    unique_classes.extend(class_names)

    # Copy images and annotations to the new directory
    for image in coco_data['images']:
        image_file_name = image['file_name']
        # Copy image to the new directory
        shutil.copy(os.path.join(dataset_path, image_file_name), os.path.join(sande_yolo_path, image_file_name))

        # Copy YOLO annotations to the new directory
        annotation_file = image_file_name.replace('.jpg', '.txt')
        if os.path.exists(os.path.join(output_yolo_dir, annotation_file)):
            shutil.copy(os.path.join(output_yolo_dir, annotation_file), os.path.join(sande_yolo_path, annotation_file))

# Process each dataset
for dataset in ['train', 'valid', 'test']:
    add_to_yolo_dir(dataset)

# Create a YAML file
yaml_content = {
    'train': [os.path.join(sande_yolo_path, img) for img in os.listdir(sande_yolo_path) if img.endswith('.jpg')],
    'val': [],
    'test': [],
    'nc': len(set(unique_classes)),
    'names': unique_classes
}

# Save the YAML file
with open(os.path.join(sande_yolo_path, 'data.yaml'), 'w') as yaml_file:
    yaml.dump(yaml_content, yaml_file, default_flow_style=False)

print("All images and annotations have been saved to 'sande_yolo' and YAML file created.")


FileNotFoundError: [Errno 2] No such file or directory: '/content/train/_annotations.coco.json'

###project_1

In [48]:
import os
import json
import shutil
import yaml

# Define the base path for project1
base_path = '/content/project1'
project1_yolo_path = os.path.join(base_path, 'project1_yolo')

# Create project1_yolo directory
os.makedirs(project1_yolo_path, exist_ok=True)

# Initialize list to hold all unique class names
unique_classes = []

# Function to convert COCO to YOLO format for project1
def coco_to_yolo(coco_json_path, image_dir, output_dir, class_names):
    # Read the COCO JSON file
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Create output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)

    # Mapping of category id to class id
    category_map = {category['id']: idx for idx, category in enumerate(class_names)}

    # Process each image
    for image in coco_data['images']:
        image_id = image['id']
        image_file_name = image['file_name']
        width = image['width']
        height = image['height']

        # Prepare YOLO annotation list
        yolo_annotations = []

        # Find corresponding annotations for this image
        for annotation in coco_data['annotations']:
            if annotation['image_id'] == image_id:
                category_id = annotation['category_id']
                if category_id in category_map:
                    # Get YOLO format (center_x, center_y, width, height)
                    bbox = annotation['bbox']
                    x_center = (bbox[0] + bbox[2] / 2) / width
                    y_center = (bbox[1] + bbox[3] / 2) / height
                    w = bbox[2] / width
                    h = bbox[3] / height
                    class_id = category_map[category_id]
                    yolo_annotations.append(f"{class_id} {x_center} {y_center} {w} {h}")

        # Write YOLO annotations to a text file
        if yolo_annotations:
            with open(os.path.join(output_dir, image_file_name.replace('.jpg', '.txt')), 'w') as out_file:
                out_file.write("\n".join(yolo_annotations))

# Function to add annotations and images to the project1_yolo directory
def add_to_yolo_dir(dataset):
    dataset_path = os.path.join(base_path, dataset)
    output_yolo_dir = os.path.join(dataset_path, 'yolo_annotations')

    # Load class names from the dataset
    class_names = []
    with open(os.path.join(dataset_path, '_annotations.coco.json'), 'r') as f:
        coco_data = json.load(f)
        for category in coco_data.get('categories', []):
            if category['name'] not in class_names:
                class_names.append(category['name'])

    # Append to unique_classes
    unique_classes.extend(class_names)

    # Copy images and annotations to the new directory
    for image in coco_data['images']:
        image_file_name = image['file_name']
        # Copy image to the new directory
        shutil.copy(os.path.join(dataset_path, image_file_name), os.path.join(project1_yolo_path, image_file_name))

        # Copy YOLO annotations to the new directory
        annotation_file = image_file_name.replace('.jpg', '.txt')
        if os.path.exists(os.path.join(output_yolo_dir, annotation_file)):
            shutil.copy(os.path.join(output_yolo_dir, annotation_file), os.path.join(project1_yolo_path, annotation_file))

# Define class names from the provided category info
class_names = [
    {"id": 0, "name": "Proj1"},
    {"id": 1, "name": "Autre"},
    {"id": 2, "name": "Metal"},
    {"id": 3, "name": "Plastique"},
    {"id": 4, "name": "carton"},
    {"id": 5, "name": "verre"},
]

# Convert each dataset
for dataset in ['train', 'valid', 'test']:
    coco_file_path = os.path.join(base_path, dataset, '_annotations.coco.json')
    image_dir_path = os.path.join(base_path, dataset)
    output_dir_path = os.path.join(image_dir_path, 'yolo_annotations')

    coco_to_yolo(coco_file_path, image_dir_path, output_dir_path, class_names)
    add_to_yolo_dir(dataset)

# Create a YAML file
yaml_content = {
    'train': [os.path.join(project1_yolo_path, img) for img in os.listdir(project1_yolo_path) if img.endswith('.jpg')],
    'val': [],
    'test': [],
    'nc': len(set(unique_classes)),
    'names': unique_classes
}

# Save the YAML file
with open(os.path.join(project1_yolo_path, 'data.yaml'), 'w') as yaml_file:
    yaml.dump(yaml_content, yaml_file, default_flow_style=False)

print("All images and annotations have been saved to 'project1_yolo' and YAML file created.")


All images and annotations have been saved to 'project1_yolo' and YAML file created.


###premba

In [ ]:
import json

# Define the path to your JSON file
json_file_path = '/content/premba/train/

# Load and print the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)  # Load the JSON data from the file
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data


In [ ]:
import os
import json
import shutil
import yaml

# Define the base path for project1
base_path = '/content/project1'
project1_yolo_path = os.path.join(base_path, 'project1_yolo')

# Create project1_yolo directory
os.makedirs(project1_yolo_path, exist_ok=True)

# Initialize list to hold all unique class names
unique_classes = []

# Function to convert COCO to YOLO format for project1
def coco_to_yolo(coco_json_path, image_dir, output_dir, class_names):
    # Read the COCO JSON file
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Create output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)

    # Mapping of category id to class id
    category_map = {category['id']: idx for idx, category in enumerate(class_names)}

    # Process each image
    for image in coco_data['images']:
        image_id = image['id']
        image_file_name = image['file_name']
        width = image['width']
        height = image['height']

        # Prepare YOLO annotation list
        yolo_annotations = []

        # Find corresponding annotations for this image
        for annotation in coco_data['annotations']:
            if annotation['image_id'] == image_id:
                category_id = annotation['category_id']
                if category_id in category_map:
                    # Get YOLO format (center_x, center_y, width, height)
                    bbox = annotation['bbox']
                    x_center = (bbox[0] + bbox[2] / 2) / width
                    y_center = (bbox[1] + bbox[3] / 2) / height
                    w = bbox[2] / width
                    h = bbox[3] / height
                    class_id = category_map[category_id]
                    yolo_annotations.append(f"{class_id} {x_center} {y_center} {w} {h}")

        # Write YOLO annotations to a text file
        if yolo_annotations:
            with open(os.path.join(output_dir, image_file_name.replace('.jpg', '.txt')), 'w') as out_file:
                out_file.write("\n".join(yolo_annotations))

# Function to add annotations and images to the project1_yolo directory
def add_to_yolo_dir(dataset):
    dataset_path = os.path.join(base_path, dataset)
    output_yolo_dir = os.path.join(dataset_path, 'yolo_annotations')

    # Load class names from the dataset
    class_names = []
    with open(os.path.join(dataset_path, '_annotations.coco.json'), 'r') as f:
        coco_data = json.load(f)
        for category in coco_data.get('categories', []):
            if category['name'] not in class_names:
                class_names.append(category['name'])

    # Append to unique_classes
    unique_classes.extend(class_names)

    # Copy images and annotations to the new directory
    for image in coco_data['images']:
        image_file_name = image['file_name']
        # Copy image to the new directory
        shutil.copy(os.path.join(dataset_path, image_file_name), os.path.join(project1_yolo_path, image_file_name))

        # Copy YOLO annotations to the new directory
        annotation_file = image_file_name.replace('.jpg', '.txt')
        if os.path.exists(os.path.join(output_yolo_dir, annotation_file)):
            shutil.copy(os.path.join(output_yolo_dir, annotation_file), os.path.join(project1_yolo_path, annotation_file))

# Define class names from the provided category info
class_names = [
    {"id": 0, "name": "Proj1"},
    {"id": 1, "name": "Autre"},
    {"id": 2, "name": "Metal"},
    {"id": 3, "name": "Plastique"},
    {"id": 4, "name": "carton"},
    {"id": 5, "name": "verre"},
]

# Convert each dataset
for dataset in ['train', 'valid', 'test']:
    coco_file_path = os.path.join(base_path, dataset, '_annotations.coco.json')
    image_dir_path = os.path.join(base_path, dataset)
    output_dir_path = os.path.join(image_dir_path, 'yolo_annotations')

    coco_to_yolo(coco_file_path, image_dir_path, output_dir_path, class_names)
    add_to_yolo_dir(dataset)

# Create a YAML file
yaml_content = {
    'train': [os.path.join(project1_yolo_path, img) for img in os.listdir(project1_yolo_path) if img.endswith('.jpg')],
    'val': [],
    'test': [],
    'nc': len(set(unique_classes)),
    'names': unique_classes
}

# Save the YAML file
with open(os.path.join(project1_yolo_path, 'data.yaml'), 'w') as yaml_file:
    yaml.dump(yaml_content, yaml_file, default_flow_style=False)

print("All images and annotations have been saved to 'project1_yolo' and YAML file created.")
